In [2]:
import datetime as dt
from pathlib import Path
import pandas as pd
print("done!")

done!


In [4]:
from model.features import (
    download_or_load_prices,
    compute_features,
    data_prep_and_feature_engineering,
)
from model.grid_search import run_grid_search
from model.backtest import run_backtest

print("downloaded!!")

downloaded!!


In [9]:
start = dt.date(2015, 1, 1)
end   = dt.date(2024, 7, 1)

# --- download SPY close prices ---
spy = download_or_load_prices(
    ["SPY"],
    Path("spy_cache.parquet"),
    start,
    end
)["Close"].rename("SPY")

# --- download ^VIX close prices ---
vix = download_or_load_prices(
    ["^VIX"],
    Path("vix_cache.parquet"),
    start,
    end
)["Close"].rename("^VIX")

# --- combine into one DataFrame with plain columns ---
market = pd.concat([spy, vix], axis=1)

print("market columns ->", market.columns.tolist())   # should be ['SPY', '^VIX']



C:\Users\17012\Desktop\market-predictions\model\features.py:27: FutureWarning: YF.download() has changed argument auto_adjust default to True
  raw = yf.download(
C:\Users\17012\Desktop\market-predictions\model\features.py:27: FutureWarning: YF.download() has changed argument auto_adjust default to True
  raw = yf.download(


market columns -> ['SPY', '^VIX']


In [10]:
# 1. raw prices for your ticker
df_prices = download_or_load_prices(
    ["AAPL"],
    Path("price_cache.parquet"),
    start,
    end
)

# 2. compute all features + target
df_feat = compute_features(df_prices, market)

# 3. build inputs for the next helper
feature_list = [c for c in df_feat.columns if c != "Target"]  # list of feature names
tickers      = ["AAPL"]                                       # list of tickers

# 4. prepare training data (function returns X_train_sel, y_train)
X_train, y_train = data_prep_and_feature_engineering(
    tickers,
    feature_list,
    Path("feature_cache.parquet"),
    start,
    end
)

print("X_train shape:", X_train.shape, "| y_train shape:", y_train.shape)


C:\Users\17012\Desktop\market-predictions\model\features.py:27: FutureWarning: YF.download() has changed argument auto_adjust default to True
  raw = yf.download(
C:\Users\17012\Desktop\market-predictions\model\features.py:91: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df["VIX_Change"] = vix.pct_change(5)
C:\Users\17012\Desktop\market-predictions\model\features.py:39: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  lvl0 = pd.to_datetime(df.index.get_level_values(0), errors="coerce")
C:\Users\17012\Desktop\market-predictions\model\features.py:27: FutureWarning: YF.download() has changed argument auto_adjust default to True
  raw = yf.download(


X_train shape: (1860, 23) | y_train shape: (1860,)


In [11]:
# hyper‑parameter search
best_params, best_model = run_grid_search(X_train, y_train)
print("best params:", best_params)

# simple back‑test (use your own test split or walk‑forward logic if you prefer)
results = run_backtest(best_model, X_train, y_train)
results.head()


[I 2025-07-28 14:13:54,384] A new study created in memory with name: no-name-5fcf16ee-ff7b-4b99-9ad5-c5255513b666


⏳  Running Optuna grid search (50 trials, CPU/GPU‑safe) …


C:\Users\17012\Desktop\market-predictions\model\grid_search.py:19: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
Positional arguments ['self', 'name', 'low', 'high', 'step', 'log'] in suggest_int() have been deprecated since v3.5.0. They will be replaced with the corresponding keyword arguments in v5.0.0, so please use the keyword specification instead. See https://github.com/optuna/optuna/releases/tag/v3.5.0 for details.
  n_estimators=trial.suggest_int("n", 100, 500, 100),
C:\Users\17012\Desktop\market-predictions\model\grid_search.py:29: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
Positional arguments ['self', 'name', 'low', 'high', 'step', 'log'] in suggest_int() have been deprecated since v3.5.0. They will be replaced with the corresponding keyword arguments in v5.0.0, so please use the keyword specification instead. See https:

[LightGBM] [Info] Number of positive: 512, number of negative: 976
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 5865
[LightGBM] [Info] Number of data points in the train set: 1488, number of used features: 23
[LightGBM] [Info] Using GPU Device: gfx902, Vendor: Advanced Micro Devices, Inc.
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 23 dense feature groups (0.03 MB) transferred to GPU in 0.004485 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.344086 -> initscore=-0.645138
[LightGBM] [Info] Start training from score -0.645138
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[2]	valid_0's binary_logloss: 0.627489


[I 2025-07-28 14:14:07,273] Trial 0 finished with value: 0.0 and parameters: {'n': 100, 'd': 13, 'leaf': 3, 'xgb_n': 400, 'xgb_lr': 0.03603652913973799, 'xgb_subsample': 0.5814318418339215, 'lgb_n': 250, 'lgb_lr': 0.0288432238641827, 'lgb_subsample': 0.7736652490153464}. Best is trial 0 with value: 0.0.


...

C:\Users\17012\Desktop\market-predictions\model\grid_search.py:19: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
Positional arguments ['self', 'name', 'low', 'high', 'step', 'log'] in suggest_int() have been deprecated since v3.5.0. They will be replaced with the corresponding keyword arguments in v5.0.0, so please use the keyword specification instead. See https://github.com/optuna/optuna/releases/tag/v3.5.0 for details.
  n_estimators=trial.suggest_int("n", 100, 500, 100),
C:\Users\17012\Desktop\market-predictions\model\grid_search.py:29: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
Positional arguments ['self', 'name', 'low', 'high', 'step', 'log'] in suggest_int() have been deprecated since v3.5.0. They will be replaced with the corresponding keyword arguments in v5.0.0, so please use the keyword specification instead. See https:

Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[3]	valid_0's binary_logloss: 0.627947


[I 2025-07-28 14:17:57,739] Trial 49 finished with value: 0.0 and parameters: {'n': 100, 'd': 13, 'leaf': 2, 'xgb_n': 500, 'xgb_lr': 0.01964747816319229, 'xgb_subsample': 0.765638069000505, 'lgb_n': 400, 'lgb_lr': 0.03665491667852435, 'lgb_subsample': 0.5388532465338416}. Best is trial 0 with value: 0.0.


Trial failed due to Must have at least 1 validation dataset for early stopping.; returning 0

===== GridSearch Results (sorted by precision) =====
 value  params_d  params_leaf  params_lgb_lr  params_lgb_n  params_lgb_subsample  params_n  params_xgb_lr  params_xgb_n  params_xgb_subsample
   0.0        13            3       0.028843           250              0.773665       100       0.036037           400              0.581432
   0.0         5            4       0.019914           200              0.781939       400       0.034524           300              0.546694
   0.0        10            4       0.059499           350              0.616808       300       0.030262           100              0.780393
   0.0        12            3       0.041000           150              0.829959       100       0.021781           350              0.582367
   0.0        14            4       0.013384           200              0.730116       500       0.079688           450              0.697748
 

TypeError: cannot unpack non-iterable NoneType object

In [13]:
# Cell GS (long‑running but silent)
import contextlib, io

buf = io.StringIO()                       # buffer to swallow stdout/stderr
with contextlib.redirect_stdout(buf), contextlib.redirect_stderr(buf):
    run_grid_search(X_train, y_train)     # no unpacking


[I 2025-07-28 16:23:26,877] A new study created in memory with name: no-name-5371892f-0f14-4a65-b08a-7ef652c09eef
[I 2025-07-28 16:23:30,984] Trial 0 finished with value: 0.0 and parameters: {'n': 100, 'd': 7, 'leaf': 1, 'xgb_n': 100, 'xgb_lr': 0.08399494596303467, 'xgb_subsample': 0.77458600737029, 'lgb_n': 350, 'lgb_lr': 0.03300694806652729, 'lgb_subsample': 0.912411387241534}. Best is trial 0 with value: 0.0.
[I 2025-07-28 16:23:35,096] Trial 1 finished with value: 0.0 and parameters: {'n': 300, 'd': 14, 'leaf': 1, 'xgb_n': 450, 'xgb_lr': 0.1581212158920422, 'xgb_subsample': 0.7052248777190286, 'lgb_n': 300, 'lgb_lr': 0.12416320488073383, 'lgb_subsample': 0.9025618918674194}. Best is trial 0 with value: 0.0.
[I 2025-07-28 16:23:39,217] Trial 2 finished with value: 0.0 and parameters: {'n': 500, 'd': 8, 'leaf': 1, 'xgb_n': 300, 'xgb_lr': 0.012773197281132562, 'xgb_subsample': 0.8735412918684983, 'lgb_n': 400, 'lgb_lr': 0.04754392493024487, 'lgb_subsample': 0.6709797770242928}. Best i

In [32]:
# ---------------------------------------------------------------
# Cell: build best model ➜ create artefact ➜ vectorbt back‑test
# ---------------------------------------------------------------
import pandas as pd, joblib, numpy as np
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost  import XGBClassifier

# --- sanity checks ------------------------------------------------
assert "X_train" in globals() and "y_train" in globals(), "Run feature‑eng cell first"
assert "df_prices" in globals(), "df_prices must be loaded"

TICKER = "AAPL"   # change if you trained on another symbol

# --- 1. load Optuna table & pull best params ----------------------
best_row = (
    pd.read_parquet("grid_search_results.parquet")
      .sort_values("value", ascending=False)
      .iloc[0]
)
best_params = {k.replace("params_", ""): best_row[k]
               for k in best_row.index if k.startswith("params_")}
print("✅ best params:", best_params)

# --- 2. fit the winning model ------------------------------------
if any(k.startswith("lgb_") for k in best_row.index):
    clean = {k.replace("lgb_", ""): v
             for k, v in best_params.items() if k.startswith("lgb_")}
    best_model = LGBMClassifier(**clean).fit(X_train, y_train)

elif any(k.startswith("xgb_") for k in best_row.index):
    clean = {k.replace("xgb_", ""): v
             for k, v in best_params.items() if k.startswith("xgb_")}
    best_model = XGBClassifier(use_label_encoder=False,
                               eval_metric="logloss",
                               **clean).fit(X_train, y_train)
else:
    rf_keys = RandomForestClassifier().get_params()
    clean   = {k: v for k, v in best_params.items() if k in rf_keys}
    best_model = RandomForestClassifier(**clean).fit(X_train, y_train)

print("🔥 model fitted")

# --- 3. locate the date level of the MultiIndex ------------------
for lev in range(X_train.index.nlevels):
    if np.issubdtype(X_train.index.get_level_values(lev).dtype, np.datetime64):
        idx_dates = X_train.index.get_level_values(lev)
        break
else:
    raise RuntimeError("Could not find datetime level in X_train index")

# --- 4. build bt_data --------------------------------------------
proba  = best_model.predict_proba(X_train)[:, 1]
signal = (proba > 0.5).astype(int)

close_px = (df_prices["Close"]
            .reindex(idx_dates)
            .ffill()
            .values)

bt_data = pd.DataFrame({
        "Date"       : idx_dates,          # already datetime64
        "Symbol"     : TICKER,
        "Close"      : close_px,
        "Predicted"  : signal,
        "Probability": proba
})

# --- 5. save artefact --------------------------------------------
art_path = Path("best_model.joblib")
artefact = {
    "model"    : best_model,
    "features" : X_train.columns.tolist(),
    "bt_data"  : bt_data
}
joblib.dump(artefact, art_path)
print("📦 artefact saved to", art_path)

# --- 6. run vectorbt back‑test -----------------------------------
results = run_backtest(art_path)      # helper expects only the Path
results.head()


✅ best params: {'d': 7, 'leaf': 1, 'lgb_lr': 0.03300694806652729, 'lgb_n': 350, 'lgb_subsample': 0.912411387241534, 'n': 100, 'xgb_lr': 0.08399494596303467, 'xgb_n': 100, 'xgb_subsample': 0.77458600737029}
🔥 model fitted


ValueError: cannot include dtype 'M' in a buffer

In [ ]:
print("---------------------------------------")